In [161]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, Point, MultiPoint
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp
from numpy.random import rand, seed
import folium
import random 
pd.options.mode.chained_assignment = None  # default='warn'

# Model inputs

## Get user input

In [2]:
hub_type = 'centralized' # none, centralized, decentralized

material_input = {
    'biobased_percentage': 'semi', # conventional, semi, full
    'circularity_percentage': 50, # 0 - 100% 
    'modularity_percentage': 20, # 0 - 100% 
}

# TRANSPORTATION INFO
transportation_input = {
    'pre-hub_transportation_type': 'road', # road, roadWater, rail
    'electrification_type': 'mixed', # diesel, mixed, electric 
}

## Make pre-determined data

In [179]:
# a10 boundary
a10 = gpd.read_file('data/A10.shp')
a10 = a10.to_crs('EPSG:28992')

# factory distances
# to add data from route planner: 
# https://www.routescanner.com/app/voyages-search
timberFactory_coords = [50.91608687148152, 5.837795943482761]
concreteFactor_coords = [51.472216701413025, 5.737978710264017]
data = {
    'material': ['timber', 'concrete'], 
    'distFromAms_km': [500, 250], # dummy number 
    'x': [timberFactory_coords[1], concreteFactor_coords[1]],
    'y': [timberFactory_coords[0], concreteFactor_coords[0]]
}
suppliers = pd.DataFrame(data)
suppliers = gpd.GeoDataFrame(suppliers, crs='EPSG:4326',
                             geometry=gpd.points_from_xy(suppliers.x, suppliers.y))
suppliers = suppliers.to_crs('EPSG:28992')
suppliers = suppliers.drop(columns=['x', 'y'])

# construction project locations and materials 
conSites_df = pd.read_csv('data/constructionSites.csv')
conSites_df = conSites_df.dropna(how='all').dropna(axis=1)
conSites_df.rename(columns={
    'Project name': 'name', 
    'Latitude': 'lat', 
    'Longitude': 'lon', 
    'Developer': 'developer', 
    'Status': 'status', 
    'Type': 'material'
}, inplace=True)

conSites_gdf = gpd.GeoDataFrame(
    conSites_df, 
    geometry=gpd.points_from_xy(conSites_df.lon, conSites_df.lat), 
    crs='EPSG:4326')
conSites_gdf = conSites_gdf.to_crs('EPSG:28992')
conSites_gdf = conSites_gdf[['name', 'geometry']]
conSites = conSites_gdf.copy() #.sample(20)

buildingType_list = ['A', 'B', 'C', 'D', 'E']
conSites['buildingType'] = np.random.choice(buildingType_list, size=len(conSites))
conSites['inA10'] = conSites.geometry.within(a10.geometry[0])

# hub locations for three scenarios: none, centralized, decentralized
minx, miny, maxx, maxy = conSites_gdf.total_bounds
candiHubs = gpd.read_file('data/candiHubs_ams.shp')
candiHubs = candiHubs.cx[minx:maxx, miny:maxy]
microHubs = candiHubs.sample(10)
microHubs['hub_type'] = 'micro'
macroHubs = pd.read_csv('data/data_construction_hubs.csv')
macroHubs = gpd.GeoDataFrame(
    macroHubs, 
    geometry=gpd.points_from_xy(macroHubs.Longitude, macroHubs.Latitude),
    crs='EPSG:4326'
)
macroHubs.rename(columns={'Name': 'name'}, inplace=True)
macroHubs = macroHubs[['name', 'geometry']]
macroHubs = macroHubs.to_crs('EPSG:28992')
macroHubs['hub_type'] = 'macro'
hubs = pd.concat([macroHubs, microHubs]).reset_index(drop=True)
hubs = hubs.reset_index(names='hub_id')
hubs = hubs[['hub_id', 'hub_type', 'geometry']]
macro = hubs[hubs.hub_type == 'macro']
hubs['nearestMacroHub'] = macro.geometry.sindex.nearest(list(hubs.geometry))[1]
hubs['inA10'] = hubs.geometry.within(a10.geometry[0])

# make dummy data for material required per building type 
data_structural = {
    'buildingType': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'E', 'E', 'E'],
    'biobased': ['none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full'],
    'concrete': [5000, 3000, 1000, 5000, 3000, 1000, 5000, 3000, 1000, 5000, 3000, 1000, 5000, 3000, 1000],
    'timber': [0, 2000, 4000, 0, 2000, 4000, 0, 2000, 4000, 0, 2000, 4000, 0, 2000, 4000],
}

data_nonStructural = {
    'buildingType': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'E', 'E', 'E'],
    'biobased': ['none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full'],
    'concrete': [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50],
    'timber': [50, 400, 600, 50, 400, 600, 50, 400, 600, 50, 400, 600, 50, 400, 600],
}
materialPerBiobasedType_structural = pd.DataFrame(data_structural)
materialPerBiobasedType_structural['materialType'] = 'structural'
materialPerBiobasedType_nonstructural = pd.DataFrame(data_nonStructural)
materialPerBiobasedType_nonstructural['materialType'] = 'nonstructural'
material_perBiobasedType = pd.concat([materialPerBiobasedType_structural, materialPerBiobasedType_nonstructural])
for mat in ['concrete', 'timber']: 
    material_perBiobasedType[mat] = material_perBiobasedType[mat].map(lambda x: x+random.randint(50,500))
    
# to add: demolition site locations 
# to add: street network

# Define functions

In [227]:
class ConstructionSites: 
    def __init__(self, gdf, biobased_type, circular_percentage, modularity_type): 
        self.biobased_type = biobased_type
        self.circular_percentage = circular_percentage
        self.modularity_type = modularity_type
        self.gdf = gdf
        
    def calc_materialsRequired(self, material_perBiobasedType): 
        biobased_type = self.biobased_type
        conSites = self.gdf
        
        def matRequired(row): 
            buildingType = row.buildingType
            matList = ['concrete', 'timber']
            for mat_type in ['structural', 'nonstructural']: 
                mat_df = material_perBiobasedType.copy()
                mat_df = mat_df[(mat_df.biobased == biobased_type) & 
                                (mat_df.buildingType == buildingType) & 
                                (mat_df.materialType == mat_type)]
                for mat in matList: 
                    row[f'{mat_type}_{mat}_tons'] = mat_df[mat].iloc[0]
            return row
        self.gdf = conSites.apply(lambda row: matRequired(row), axis=1)
    
class Hubs: 
    def __init__(self, gdf, hub_network):
        self.hub_network = hub_network
        self.gdf = gdf
        
class Model(ConstructionSites, Hubs): 
    def __init__(self):
        None
        
    def load_data(self, constructionSites_gdf, hubs_gdf, material_perBuildingType_df, suppliers_df): 
        self.constructionSites = constructionSites_gdf
        self.hubs = hubs_gdf
        self.material_perBuildingType = material_perBuildingType_df
        self.suppliers = suppliers_df
    
    def load_parameters(self, biobased_type, circular_percentage, modularity_type, 
                        hub_network, transportation_network, vehicle_type):
        
        # initialize ConstructionSites and Hubs
        conSites = ConstructionSites(self.constructionSites, biobased_type, circular_percentage, modularity_type)
        conSites.calc_materialsRequired(self.material_perBuildingType)
        self.constructionSites = conSites.gdf
        self.hubs = Hubs(self.hubs, hub_network).gdf
        
        # add parameters
        self.circular_percentage = circular_percentage
        self.modularity_type = modularity_type
        self.transportation_network = transportation_network
        self.vehicle_type = vehicle_type
        self.hub_network = hub_network
    
    def run_model(self): 
        self.find_nearestHubs()
        self.calc_distances()
        self.calc_emissions_h2c()
        self.calc_emissions_s2h()
        self.calc_emissions_total()
        
    def find_nearestHubs(self): 
        conSites = self.constructionSites
        hubs = self.hubs 
        macroHubs = hubs[hubs.hub_type == 'macro']
        conSites['nearestHub'] = hubs.geometry.sindex.nearest(list(conSites.geometry))[1]
        conSites['nearestMacroHub'] = macroHubs.geometry.sindex.nearest(list(conSites.geometry))[1]
        self.constructionSites = conSites
        
    def calc_distances(self): 
        conSites = self.constructionSites
        hubs = self.hubs 
        
        def calc_dists_cons2hub(row): 
            macroHubs = hubs[hubs.hub_type == 'macro']
            conGeom = row.geometry
            hubGeom = hubs[hubs.hub_id == row.nearestHub].geometry.iloc[0]
            hubGeom_macro = macroHubs[macroHubs.hub_id == row.nearestMacroHub].geometry.iloc[0]
            row['dist_micro2cons'] = conGeom.distance(hubGeom)
            row['dist_macro2cons'] = conGeom.distance(hubGeom_macro)
            row['dist_macro2micro'] = hubGeom_macro.distance(hubGeom)
            return row 
        conSites = conSites.apply(lambda row: calc_dists_cons2hub(row), axis=1)
        self.constructionSites = conSites
    
    def calc_nTrucks(self): 
        # calculate number of trucks needed for each site 
        # according to materials and modularity
        None
    
    def calc_emissions_h2c(self): 
        conSites = self.constructionSites
        hubs = self.hubs
        vehicle_type = self.vehicle_type
        hub_network = self.hub_network 
        
        emissionsPerKm_tons_diesel = 0.9/1000 
        emissionsPerKm_tons_electric = 0
        truckCapacity_tons = 25 
        truckFill_perc = 0.8
        truckCapacity_tons = truckCapacity_tons * truckFill_perc

        def calc_emissions_h2c_lambda(row): 
            mat_tons = row.concrete_tons + row.timber_tons
            nTrucks = math.ceil(mat_tons / truckCapacity_tons)
            
            conSite_inA10 = row.inA10
            hub_inA10 = hubs[hubs.hub_id == row.nearestHub].inA10.iloc[0]
            
            dist_macro2micro = row.dist_macro2micro / 1000
            dist_micro2c = row.dist_micro2cons / 1000
            dist_tot = (row.dist_micro2cons + row.dist_macro2micro) / 1000
            
            # calc emissions based on conditions
            if vehicle_type == 'diesel': 
                row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_diesel
            elif vehicle_type == 'semi': 
                if hub_network == 'decentralized': 
                    if hub_inA10: 
                        row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
                    else: # hub outside A10  
                        emissions_macro2micro = dist_macro2micro * nTrucks * emissionsPerKm_tons_diesel
                        if conSite_inA10: 
                            emissions_micro2cons = dist_micro2c * nTrucks * emissionsPerKm_tons_electric
                        else: # consite outside A10 
                            emissions_micro2cons = dist_micro2c * nTrucks * emissionsPerKm_tons_diesel
                        row['emissions_h2c'] = emissions_macro2micro + emissions_micro2cons
                elif hub_network == 'centralized': 
                    if conSite_inA10:
                        row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
                    else: # conSite outside A10: 
                        row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_diesel
            elif vehicle_type == 'electric': 
                row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
            if hub_network == 'none': 
                row['emissions_h2c'] = 0 
                
            return row       
        conSites = conSites.apply(lambda row: calc_emissions_h2c_lambda(row), axis=1)
        self.constructionSites = conSites
        self.emissions_h2c = conSites.emissions_h2c.sum()
        
    def calc_emissions_s2h(self): 
        conSites = self.constructionSites
        hubs = self.hubs
        transportation_network = self.transportation_network
        hub_network = self.hub_network
        transportation_network_stats = {
            'road': {'capacity': 25, 'emissions': 0.9/1000},
            'roadWater': {'capacity': 50, 'emissions': 0.9/1000},
            'rail': {'capacity': 60, 'emissions': 0.9/1000}
        }
        capacity_tons = transportation_network_stats[transportation_network]['capacity']
        emissionsPerKm_tons = transportation_network_stats[transportation_network]['emissions']
        
        if hub_network == 'none': 
            if transportation_network != 'road': 
                print('''if the 'no hubs' scenario is chosen, road network is the only option.''')
            hubs = conSites[['name', 'geometry', 'concrete_tons', 'timber_tons']]
            hubs.rename(columns={'name': 'hub_id'}, inplace=True)
            fill_perc = 0.4
        else: # hub_network == 'centralized' or 'decentralized'
            matRequired = conSites.groupby(by='nearestMacroHub').sum(numeric_only=True).reset_index()
            matRequired.rename(columns={'nearestMacroHub': 'hub_id'}, inplace=True)
            hubs = hubs.merge(matRequired, on='hub_id')
            hubs = hubs[['hub_id', 'concrete_tons', 'timber_tons', 'geometry']]
            fill_perc = 0.8 
        capacity_tons = capacity_tons * fill_perc
        
        def calc_emissions_s2h_lambda(row): 
            distList = []
            for mat in ['concrete', 'timber']: 
                mat_tons = row[f'{mat}_tons'] 
                nTrucks = math.ceil(mat_tons / capacity_tons) 
                dist = suppliers[suppliers.material == mat].distFromAms_km.iloc[0] * nTrucks
                distList.append(dist)
            row['emissions_s2h_tons'] = sum(distList) * emissionsPerKm_tons
            return row 
        hubs = hubs.apply(lambda row: calc_emissions_s2h_lambda(row), axis=1)
        self.emissions_s2h = hubs.emissions_s2h_tons.sum()
    
    def calc_emissions_total(self): 
        self.emissions_total = self.emissions_s2h + self.emissions_h2c
        
    def visualize(self, zoom_start=12): 
        self.make_gdfs_forPlotting()
        self.make_coords_forPlotting()
        
        m = folium.Map([52.377231, 4.899288], zoom_start=zoom_start, tiles='cartodbdark_matter')
        self.folium_plotPolygons(m, a10, 'grey')
        if self.hub_network == 'none': 
            folium.PolyLine(locations=self.coords_supplier2site, weight=1, color='#4a4a4a', dash_array='5').add_to(m)
        elif self.hub_network == 'centralized': 
            folium.PolyLine(locations=self.coords_supplier2macro, weight=1, color='#4a4a4a', dash_array='5').add_to(m)
            folium.PolyLine(locations=self.coords_macro2site, weight=1, color='white', dash_array='5').add_to(m)
        elif self.hub_network == 'decentralized': 
            folium.PolyLine(locations=self.coords_supplier2macro, weight=1, color='#4a4a4a', dash_array='5').add_to(m)
            folium.PolyLine(locations=self.coords_macro2micro, weight=1, color='white', dash_array='5').add_to(m)
            folium.PolyLine(locations=self.coords_micro2site, weight=1, color='white', dash_array='5').add_to(m)
        self.folium_plotPoints(m, self.macroHubs_plot, 'red', radius=5)
        self.folium_plotPoints(m, self.microHubs_plot, 'red')
        self.folium_plotPoints(m, self.conSites_plot, 'lightblue')
        return m

    def make_gdfs_forPlotting(self): 
        self.hubs_plot = self.hubs.to_crs('EPSG:4326')
        self.conSites_plot = self.constructionSites[['geometry', 'nearestHub', 'nearestMacroHub']].to_crs('EPSG:4326')
        self.macroHubs_plot = self.hubs_plot[hubs_plot.hub_type == 'macro']
        self.microHubs_plot = self.hubs_plot[hubs_plot.hub_type == 'micro']
        self.suppliers_plot = self.suppliers.to_crs('EPSG:4326')
        
    def make_coords_forPlotting(self): 
        conSites_plot = self.conSites_plot
        hubs_plot = self.hubs_plot
        macroHubs_plot = self.microHubs_plot
        self.coords_supplier2site = self.makeCoords_lines_supplier2dest(conSites_plot)
        self.coords_supplier2macro = self.makeCoords_lines_supplier2dest(macroHubs_plot)
        self.coords_macro2site = self.makeCoords_lines_hub2cons(hub_type='macro')
        self.coords_micro2site = self.makeCoords_lines_hub2cons(hub_type='micro')
        self.coords_macro2micro = self.makeCoords_lines_macro2micro()
        
    def folium_plotPoints(self, m, gdf, color, radius=1):
        gdf = gdf.to_crs('EPSG:4326')
        gdf['x'] = gdf.geometry.x
        gdf['y'] = gdf.geometry.y
        locList = gdf[['y', 'x']].values.tolist()
        for loc in locList:
            folium.CircleMarker(location=loc, radius=radius, fill_color=color, color=color).add_to(m)

    def folium_plotPolygons(self, m, gdf, color): 
        gdf = gdf.to_crs('EPSG:4326')
        coords = list(gdf.geometry[0].exterior.coords)
        coords = [[x[1], x[0]] for x in coords]
        folium.Polygon(locations = coords, color=color,
                       fill_color=color, fill_opacity=0.2, fill=True).add_to(m)
        
    def geom_to_coordsList(self, geometrySeries):
        geometrySeries = gpd.GeoSeries(geometrySeries)
        coords_list = list(zip(list(geometrySeries.y), list(geometrySeries.x)))
        coords_list = [list(x) for x in coords_list]
        return coords_list
    
    def makeCoords_lines_hub2cons(self, hub_type):
        hubs_plot=self.hubs_plot
        conSites_plot=self.conSites_plot
        
        left_on = 'nearestHub' if hub_type == 'micro' else 'nearestMacroHub'
        temp = conSites_plot.merge(hubs_plot, left_on=left_on, right_on='hub_id')
        temp.rename(columns={'geometry_x': 'geom_con', 'geometry_y': 'geom_hub'}, inplace=True)
        temp = temp[['geom_con', 'geom_hub']]

        coord_con = self.geom_to_coordsList(temp.geom_con)
        coord_hub = self.geom_to_coordsList(temp.geom_hub)
        coord_list = list(zip(coord_con, coord_hub))
        coord_list = [list(x) for x in coord_list]
        return coord_list

    def makeCoords_lines_macro2micro(self):
        hubs_plot=self.hubs_plot
        macroHubs_plot = hubs_plot[hubs_plot.hub_type == 'macro']
        microHubs_plot = hubs_plot[hubs_plot.hub_type == 'micro']
        t = microHubs_plot.merge(macroHubs_plot, left_on='nearestMacroHub', right_on='hub_id')
        t.rename(columns={'geometry_x': 'geom_micro', 'geometry_y': 'geom_macro'}, inplace=True)
        t = t[['geom_micro', 'geom_macro']]

        coord_micro = self.geom_to_coordsList(t.geom_micro)
        coord_macro = self.geom_to_coordsList(t.geom_macro)
        coord_list = list(zip(coord_micro, coord_macro))
        coord_list = [list(x) for x in coord_list]
        return coord_list

    def makeCoords_lines_supplier2dest(self, dest_plot):
        suppliers_plot=self.suppliers_plot
        coord_list = []
        for material in ['timber', 'concrete']: 
            supplier_geom = self.geom_to_coordsList(suppliers_plot[suppliers_plot.material==material].geometry)[0]
            supplier_coords = [supplier_geom for i in range(len(dest_plot))]
            dest_coords = self.geom_to_coordsList(dest_plot.geometry)
            coord_list_mat = list(zip(supplier_coords, dest_coords))
            coord_list_mat = [list(x) for x in coord_list_mat]
            coord_list.extend(coord_list_mat)
        return coord_list

In [ ]:
# calculate emissions separately for structural and non-structural elements 
    # vary the following params depending on modularity_type: 
        # conSites - structural, non-structural, all 
        # truckFill_tons - lower when transporting modules (weight of 2 modules)

In [214]:
biobased_type = 'none' # none, semi, full 
circular_percentage = 50
modularity_type = 'full' # none, full 

model = Model()
model.load_data(conSites, hubs, material_perBiobasedType, suppliers)
model.load_parameters(biobased_type='full', circular_percentage=50, modularity_type='semi', 
                  hub_network='centralized', transportation_network='road', vehicle_type='diesel')
model.find_nearestHubs()
model.calc_distances()
c = model.constructionSites
h = model.hubs

In [219]:
h.head()

,hub_id,hub_type,geometry,nearestMacroHub,inA10
0,0,macro,POINT (117482.212 492252.729),0,False
1,1,macro,POINT (126315.461 480465.506),1,False
2,2,micro,POINT (119512.204 488153.957),0,True
3,3,micro,POINT (125020.223 483277.438),1,False
4,4,micro,POINT (121917.655 489519.124),0,True


In [220]:
conSites.head()

,name,geometry,buildingType,inA10
0,Haut,POINT (121490.453 487040.018),E,True
1,Switi,POINT (126621.374 480391.719),B,False
2,Stories,POINT (122104.436 489910.504),D,True
3,Patch 22,POINT (122207.920 489821.802),E,True
4,Top-up,POINT (122241.078 489812.903),D,True


In [69]:
# run model
model = Model()
model.load_data(conSites, hubs, material_perBuildingType, suppliers)
model.load_parameters(biobased_type='full', circular_percentage=50, modularity_type='semi', 
                  hub_network='centralized', transportation_network='road', vehicle_type='diesel')
model.run_model()
model.visualize(zoom_start=11)

In [221]:
suppliers

,material,distFromAms_km,geometry
0,timber,500,POINT (186688.613 325245.792)
1,concrete,250,POINT (179371.616 387076.343)


In [217]:
c

,name,geometry,buildingType,inA10,structural_concrete_tons,structural_timber_tons,nonstructural_concrete_tons,nonstructural_timber_tons,nearestHub,nearestMacroHub,dist_micro2cons,dist_macro2cons,dist_macro2micro
0,Haut,POINT (121490.453 487040.018),E,True,1492,4465,130,702,2,0,2270.314547,6575.587927,4573.926465
1,Switi,POINT (126621.374 480391.719),B,False,1192,4354,140,811,1,1,314.686153,314.686153,0.000000
2,Stories,POINT (122104.436 489910.504),D,True,1296,4494,469,1053,4,0,433.665608,5181.792220,5210.158934
3,Patch 22,POINT (122207.920 489821.802),E,True,1492,4465,130,702,4,0,419.365595,5314.294076,5210.158934
4,Top-up,POINT (122241.078 489812.903),D,True,1296,4494,469,1053,4,0,436.930234,5347.855313,5210.158934
5,Horizons Amsterdam,POINT (125910.987 487616.724),A,True,1301,4082,454,950,7,1,1812.099011,7162.647710,5786.566814
6,Patchwork,POINT (125910.987 487616.724),D,True,1296,4494,469,1053,7,1,1812.099011,7162.647710,5786.566814
7,Hotel Jakarta,POINT (125910.987 487616.724),D,True,1296,4494,469,1053,7,1,1812.099011,7162.647710,5786.566814
8,Stepstone,POINT (119278.132 483373.890),D,False,1296,4494,469,1053,9,1,3719.089640,7614.636464,4325.655153
9,The Valley,POINT (120256.141 483464.052),D,False,1296,4494,469,1053,9,1,2747.934361,6760.668531,4325.655153


In [222]:
suppliers

,material,distFromAms_km,geometry
0,timber,500,POINT (186688.613 325245.792)
1,concrete,250,POINT (179371.616 387076.343)


In [ ]:
# if modularity_type == 'full': 
    # structural truck capacity = 25 
    # non structural truck capacity = 10
# elif modularity_type == 'none': 
    # both structural and non-structural truck capacity = 25 
# for each construction site: 
    # calculate nTrucks required for structural materials according to capacity
    # calculate nTrucks required for non-structural materials according to capacity

In [212]:
def calc_nTrucks(row, modularity_type): 
    # truck capacity per modularity type 
    truckCapacity_dict = {
        'full': {'structural': 25, 'nonstructural': 10}, 
        'none': {'structural': 25, 'nonstructural': 25}
    } 
    
    # calc matRequired for structural and nonstructural materials 
    matRequired_dict = {}
    for struc_type in ['structural', 'nonstructural']: 
        matRequired = sum([row[f'{struc_type}_{mat}_tons']for mat in ['concrete', 'timber']])
        matRequired_dict[struc_type] = matRequired
        
    # calc nTrucks according to modularity_type
    nTrucks_dict = {}
    for struc_type in ['structural', 'nonstructural']: 
        nTrucks = matRequired_dict[struc_type]
    nTrucks_structural = matRequired_dict['structural'] / truckCapacity_dict[modularity_type]['structural']
    nTrucks_nonstructural = 
    nTrucks = nTrucks_structural + nTrucks_nonstructural
    
    # calc nTrucks 
    row['nTrucks'] = math.ceil(matRequired / truckCapacity)
    return row
c.apply(lambda row: calc_nTrucks(row, modularity_type), axis=1)

,name,geometry,buildingType,inA10,structural_concrete_tons,structural_timber_tons,nonstructural_concrete_tons,nonstructural_timber_tons,nearestHub,nearestMacroHub,dist_micro2cons,dist_macro2cons,dist_macro2micro,nTrucks
0,Haut,POINT (121490.453 487040.018),E,True,1492,4465,130,702,2,0,2270.314547,6575.587927,4573.926465,239
1,Switi,POINT (126621.374 480391.719),B,False,1192,4354,140,811,1,1,314.686153,314.686153,0.000000,222
2,Stories,POINT (122104.436 489910.504),D,True,1296,4494,469,1053,4,0,433.665608,5181.792220,5210.158934,232
3,Patch 22,POINT (122207.920 489821.802),E,True,1492,4465,130,702,4,0,419.365595,5314.294076,5210.158934,239
4,Top-up,POINT (122241.078 489812.903),D,True,1296,4494,469,1053,4,0,436.930234,5347.855313,5210.158934,232
5,Horizons Amsterdam,POINT (125910.987 487616.724),A,True,1301,4082,454,950,7,1,1812.099011,7162.647710,5786.566814,216
6,Patchwork,POINT (125910.987 487616.724),D,True,1296,4494,469,1053,7,1,1812.099011,7162.647710,5786.566814,232
7,Hotel Jakarta,POINT (125910.987 487616.724),D,True,1296,4494,469,1053,7,1,1812.099011,7162.647710,5786.566814,232
8,Stepstone,POINT (119278.132 483373.890),D,False,1296,4494,469,1053,9,1,3719.089640,7614.636464,4325.655153,232
9,The Valley,POINT (120256.141 483464.052),D,False,1296,4494,469,1053,9,1,2747.934361,6760.668531,4325.655153,232


In [205]:
c

,name,geometry,buildingType,inA10,structural_concrete_tons,structural_timber_tons,nonstructural_concrete_tons,nonstructural_timber_tons,nearestHub,nearestMacroHub,dist_micro2cons,dist_macro2cons,dist_macro2micro
0,Haut,POINT (121490.453 487040.018),E,True,1492,4465,130,702,2,0,2270.314547,6575.587927,4573.926465
1,Switi,POINT (126621.374 480391.719),B,False,1192,4354,140,811,1,1,314.686153,314.686153,0.000000
2,Stories,POINT (122104.436 489910.504),D,True,1296,4494,469,1053,4,0,433.665608,5181.792220,5210.158934
3,Patch 22,POINT (122207.920 489821.802),E,True,1492,4465,130,702,4,0,419.365595,5314.294076,5210.158934
4,Top-up,POINT (122241.078 489812.903),D,True,1296,4494,469,1053,4,0,436.930234,5347.855313,5210.158934
5,Horizons Amsterdam,POINT (125910.987 487616.724),A,True,1301,4082,454,950,7,1,1812.099011,7162.647710,5786.566814
6,Patchwork,POINT (125910.987 487616.724),D,True,1296,4494,469,1053,7,1,1812.099011,7162.647710,5786.566814
7,Hotel Jakarta,POINT (125910.987 487616.724),D,True,1296,4494,469,1053,7,1,1812.099011,7162.647710,5786.566814
8,Stepstone,POINT (119278.132 483373.890),D,False,1296,4494,469,1053,9,1,3719.089640,7614.636464,4325.655153
9,The Valley,POINT (120256.141 483464.052),D,False,1296,4494,469,1053,9,1,2747.934361,6760.668531,4325.655153


In [204]:
# calculate emissions from hub to construction site 
vehicle_type = 'diesel' # diesel, semi, electric
hub_network = 'centralized' # none, centralized, decentralized 
def calc_emissions_h2c_lambda(row): 
    mat_tons_list = []
    for mat in ['concrete', 'timber']: 
        mat_tons = row[f'structural_{mat}_tons']
        mat_tons_list.append(mat_tons)
    mat_tons = sum(mat_tons_list)
    nTrucks = math.ceil(mat_tons / truckCapacity_tons)

    conSite_inA10 = row.inA10
    hub_inA10 = hubs[hubs.hub_id == row.nearestHub].inA10.iloc[0]

    dist_macro2micro = row.dist_macro2micro / 1000
    dist_micro2c = row.dist_micro2cons / 1000
    dist_tot = (row.dist_micro2cons + row.dist_macro2micro) / 1000

    # calc emissions based on conditions
    if vehicle_type == 'diesel': 
        row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_diesel
    elif vehicle_type == 'semi': 
        if hub_network == 'decentralized': 
            if hub_inA10: 
                row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
            else: # hub outside A10  
                emissions_macro2micro = dist_macro2micro * nTrucks * emissionsPerKm_tons_diesel
                if conSite_inA10: 
                    emissions_micro2cons = dist_micro2c * nTrucks * emissionsPerKm_tons_electric
                else: # consite outside A10 
                    emissions_micro2cons = dist_micro2c * nTrucks * emissionsPerKm_tons_diesel
                row['emissions_h2c'] = emissions_macro2micro + emissions_micro2cons
        elif hub_network == 'centralized': 
            if conSite_inA10:
                row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
            else: # conSite outside A10: 
                row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_diesel
    elif vehicle_type == 'electric': 
        row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
    if hub_network == 'none': 
        row['emissions_h2c'] = 0 

    return row       

c.apply(lambda row: calc_emissions_h2c_lambda(row), axis=1)

NameError: name 'truckCapacity_tons' is not defined

In [69]:
# run model
model = Model()
model.load_data(conSites, hubs, material_perBuildingType, suppliers)
model.load_parameters(biobased_type='full', circular_percentage=50, modularity_type='semi', 
                  hub_network='centralized', transportation_network='road', vehicle_type='diesel')
model.run_model()
model.visualize(zoom_start=11)